In [ ]:
#%matplotlib notebook
%matplotlib inline

import matplotlib
matplotlib.rcParams['figure.figsize'] = (12, 9)

import scipy
import scipy.stats
import math
import numpy as np
import pandas as pd

#import ipywidgets
#from ipywidgets import interact

## Cut images with signal on borders

Definitions:
* border > 0 ("contenu")
* border == 0 ("non contenu")

Follow this: https://github.com/jdhp-sap/tino_cta#edge-rejection

In [ ]:
CSV_FILE_PATH = "../xps/best.csv"

#WAVELET_LABEL = "WT-K-k-C1-m3-n4-s3"
WAVELET_LABEL = "WT-K-k-C1-m3-n4-s2-2-3-3"

TAILCUT_LABEL = "Tailcut-5-10"

PART = 0         # 0 for gamma, 1 for protons

FAINT_BRIGHT_BORDER = 100

full_df = pd.read_csv(CSV_FILE_PATH)

ref = full_df[full_df.Type == 'Ref']
ref = ref[ref.Part == PART]

tc = full_df[full_df.Type == TAILCUT_LABEL]
tc = tc[tc.Part == PART]

wt = full_df[full_df.Type == WAVELET_LABEL]
wt = wt[wt.Part == PART]

tc_wt = pd.merge(tc, wt, on="Id", how="outer", suffixes=('_tc', '_wt'))  #.dropna(how='any')
df = pd.merge(tc_wt, ref, on="Id", how="outer")

df["delta_tc"] = np.fmod(((df['hPsi'] - df['hPsi_tc']) * 180. / np.pi), 90.)
df["delta_wt"] = np.fmod(((df['hPsi'] - df['hPsi_wt']) * 180. / np.pi), 90.)

df.delta_tc = abs(df.delta_tc)
df.delta_wt = abs(df.delta_wt)

dfi = df[df.border >  0]
dfo = df[df.border == 0]

In [ ]:
df.columns
#df.groupby(["Type","Part","Success"]).count()
#df.groupby(["Type","Part"]).describe()

In [ ]:
print("{} images contained, {} not contained".format(len(dfi), len(dfo)))

### NPE on cleaned image

In [ ]:
fig, ax = plt.subplots()

dfi['peSum'].plot.hist(alpha=0.2,  bottom=0.1,
                       bins=np.logspace(0.1, 6., 200),
                       label="contained ({})".format(len(dfi)), legend=True)
dfo['peSum'].plot.hist(alpha=0.2,  bottom=0.1,
                       bins=np.logspace(0.1, 6., 200),
                       label="not contained ({})".format(len(dfo)), legend=True)

ax.axvline(x=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axvline(x=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xscale('log')

ax.set_title("Ref.", fontsize=20)

CCL:

* more images on border (*not contained images*) at high energy

## Analysis of the energy estimator bias

### PEmax VS PEsum for Ref.

In [ ]:
fig, ax = plt.subplots()

ax.plot(dfi['peMax1'], dfi['peSum'], '.b', alpha=0.3, label="contained ({})".format(len(dfi)))
ax.plot(dfo['peMax1'], dfo['peSum'], '.r', alpha=0.3, label="not contained ({})".format(len(dfo)))

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel("pe max", fontsize=20)
ax.set_ylabel("pe sum", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title("PEmax VS PEsum for Ref.", fontsize=20)

In [ ]:
fig, ax = plt.subplots()

x1, y1 = dfi['peMax1'], dfi['peSum']
x2, y2 = dfo['peMax1'], dfo['peSum']

xbins = np.logspace(0, 4, 30)
ybins = np.logspace(0, 5, 30)

counts1, xedges, yedges = np.histogram2d(x1, y1, bins=(xbins, ybins))
counts2, xedges, yedges = np.histogram2d(x2, y2, bins=(xbins, ybins))

max_value = max(np.max(counts1), np.max(counts2))

#levels = np.arange(10, 60, 10)
levels = np.array([0.01*max_value, 0.3*max_value, 0.5*max_value])

cs1 = plt.contour(xedges[:-1], yedges[:-1], counts1.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='blue', label="Ref")
ax.clabel(cs1, inline=False, fontsize=12)

cs2 = plt.contour(xedges[:-1], yedges[:-1], counts2.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='red', label="Ref")
ax.clabel(cs2, inline=False, fontsize=12)

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

#ax.set_xlim(1e-2, 1e1)
#ax.set_ylim(1e0, 1e5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel("pe max", fontsize=20)
ax.set_ylabel("pe sum", fontsize=20)

lines = [ cs1.collections[0], cs2.collections[0]]
labels = ["contained ({})".format(len(dfi)), "not contained ({})".format(len(dfo))]
ax.legend(lines, labels, prop={'size': 14}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title("PEmax VS PEsum for Ref.", fontsize=20)

### nPix VS PEsum for Ref.

In [ ]:
fig, ax = plt.subplots()

ax.plot(dfi['nPix'], dfi['peSum'], '.b', alpha=0.3, label="contained ({})".format(len(dfi)))
ax.plot(dfo['nPix'], dfo['peSum'], '.r', alpha=0.3, label="not contained ({})".format(len(dfo)))

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel("num pixels > 0", fontsize=20)
ax.set_ylabel("pe sum", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title("NumPix>0 VS PEsum for Ref.", fontsize=20)

In [ ]:
fig, ax = plt.subplots()

x1, y1 = dfi['nPix'], dfi['peSum']
x2, y2 = dfo['nPix'], dfo['peSum']

xbins = np.logspace(0, 3, 30)
ybins = np.logspace(0, 5, 30)

counts1, xedges, yedges = np.histogram2d(x1, y1, bins=(xbins, ybins))
counts2, xedges, yedges = np.histogram2d(x2, y2, bins=(xbins, ybins))

max_value = max(np.max(counts1), np.max(counts2))

#levels = np.arange(10, 60, 10)
levels = np.array([0.01*max_value, 0.3*max_value, 0.5*max_value])

cs1 = plt.contour(xedges[:-1], yedges[:-1], counts1.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='blue', label="Ref")
ax.clabel(cs1, inline=False, fontsize=12)

cs2 = plt.contour(xedges[:-1], yedges[:-1], counts2.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='red', label="Ref")
ax.clabel(cs2, inline=False, fontsize=12)

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

#ax.set_xlim(1e-2, 1e1)
#ax.set_ylim(1e0, 1e5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel("num pixels > 0", fontsize=20)
ax.set_ylabel("pe sum", fontsize=20)

lines = [ cs1.collections[0], cs2.collections[0]]
labels = ["contained ({})".format(len(dfi)), "not contained ({})".format(len(dfo))]
ax.legend(lines, labels, prop={'size': 14}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title("NumPix>0 VS PEsum for Ref.", fontsize=20)

### (Hillas length / Hillas width) VS PEsum for Ref.

In [ ]:
fig, ax = plt.subplots()

ax.plot(dfi['hLength']/dfi['hWidth'], dfi['peSum'], '.b', alpha=0.3, label="contained ({})".format(len(dfi)))
ax.plot(dfo['hLength']/dfo['hWidth'], dfo['peSum'], '.r', alpha=0.3, label="not contained ({})".format(len(dfo)))

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_yscale('log')
#ax.set_xscale('log')

ax.set_xlabel("Hillas length / Hillas width", fontsize=20)
ax.set_ylabel("pe sum", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title("(Hillas length / Hillas width) VS PEsum for Ref.", fontsize=20)

In [ ]:
fig, ax = plt.subplots()

x1, y1 = dfi['hLength']/dfi['hWidth'], dfi['peSum']
x2, y2 = dfo['hLength']/dfo['hWidth'], dfo['peSum']

xbins = np.linspace(1, 7, 30)
ybins = np.logspace(0, 5, 30)

counts1, xedges, yedges = np.histogram2d(x1, y1, bins=(xbins, ybins))
counts2, xedges, yedges = np.histogram2d(x2, y2, bins=(xbins, ybins))

max_value = max(np.max(counts1), np.max(counts2))

#levels = np.arange(10, 60, 10)
levels = np.array([0.01*max_value, 0.3*max_value, 0.5*max_value])

cs1 = plt.contour(xedges[:-1], yedges[:-1], counts1.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='blue', label="Ref")
ax.clabel(cs1, inline=False, fontsize=12)

cs2 = plt.contour(xedges[:-1], yedges[:-1], counts2.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='red', label="Ref")
ax.clabel(cs2, inline=False, fontsize=12)

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

#ax.set_xlim(1e-2, 1e1)
#ax.set_ylim(1e0, 1e5)

ax.set_yscale('log')
#ax.set_xscale('log')

ax.set_xlabel("Hillas length / Hillas width", fontsize=20)
ax.set_ylabel("pe sum", fontsize=20)

lines = [ cs1.collections[0], cs2.collections[0]]
labels = ["contained ({})".format(len(dfi)), "not contained ({})".format(len(dfo))]
ax.legend(lines, labels, prop={'size': 14}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title("(Hillas length / Hillas width) VS PEsum for Ref.", fontsize=20)

### Analysis

Contained images are a bit less ellongated.

## Epsilon energy

### Tailcut

In [ ]:
fig, ax = plt.subplots()

ax.plot(dfi['Denergy_tc'], dfi['peSum_tc'], '.b', alpha=0.3, label="contained ({})".format(len(dfi)))
ax.plot(dfo['Denergy_tc'], dfo['peSum_tc'], '.r', alpha=0.3, label="not contained ({})".format(len(dfo)))

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel(r"$\mathcal{E}$ energy", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

ax.set_xlim(1e-4, 1e1)
ax.set_ylim(1e0, 1e5)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title(r"$\mathcal{E}$ energy for " + TAILCUT_LABEL, fontsize=20)

In [ ]:
fig, ax = plt.subplots()

x1, y1 = dfi['Denergy_tc'], dfi['peSum_tc']
x2, y2 = dfo['Denergy_tc'], dfo['peSum_tc']

xbins = np.logspace(-3, 1, 30)
ybins = np.logspace(0, 5, 30)

counts1, xedges, yedges = np.histogram2d(x1, y1, bins=(xbins, ybins))
counts2, xedges, yedges = np.histogram2d(x2, y2, bins=(xbins, ybins))

max_value = max(np.max(counts1), np.max(counts2))

#levels = np.arange(10, 60, 10)
levels = np.array([0.01*max_value, 0.3*max_value, 0.5*max_value])

cs1 = plt.contour(xedges[:-1], yedges[:-1], counts1.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='blue', label="TC")
ax.clabel(cs1, inline=False, fontsize=12)

cs2 = plt.contour(xedges[:-1], yedges[:-1], counts2.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='red', label="WT")
ax.clabel(cs2, inline=False, fontsize=12)

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlim(1e-2, 1e1)
ax.set_ylim(1e0, 1e5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel(r"$\mathcal{E}$ energy", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

lines = [ cs1.collections[0], cs2.collections[0]]
labels = ["contained ({})".format(len(dfi)), "not contained ({})".format(len(dfo))]
ax.legend(lines, labels, prop={'size': 14}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title(r"$\mathcal{E}$ energy for " + TAILCUT_LABEL, fontsize=20)

CCL for **Tailcut**:

1. less images at high energy for *contained images* (coherent previous ccl)
2. **better energy conservation for *contained images*** (why ?)

** Reminder **

$\mathcal{E}_{\text{energy}}$ (relative total counts difference) is defined by:

$$
\mathcal{E}_{\text{energy}}(\hat{\boldsymbol{S}}, \boldsymbol{S}^*) = \frac{ \text{abs} \left( \sum_i \hat{\boldsymbol{S}}_i - \sum_i \boldsymbol{S}^*_i \right) }{ \sum_i \boldsymbol{S}^*_i }
$$

with:

- $\hat{\boldsymbol{S}}$ the algorithm's output image (i.e. the *cleaned* image);
- $\boldsymbol{S}^*$ the reference image (i.e. the *clean* image).

### Wavelets

In [ ]:
fig, ax = plt.subplots()

ax.plot(dfi['Denergy_wt'], dfi['peSum_wt'], '.b', alpha=0.3, label="contained ({})".format(len(dfi)))
ax.plot(dfo['Denergy_wt'], dfo['peSum_wt'], '.r', alpha=0.3, label="not contained ({})".format(len(dfo)))

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel(r"$\mathcal{E}$ energy", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

ax.set_xlim(1e-4, 1e1)
ax.set_ylim(1e0, 1e5)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title(r"$\mathcal{E}$ energy for " + WAVELET_LABEL, fontsize=20)

In [ ]:
fig, ax = plt.subplots()

x1, y1 = dfi['Denergy_wt'], dfi['peSum_wt']
x2, y2 = dfo['Denergy_wt'], dfo['peSum_wt']

xbins = np.logspace(-3, 1, 30)
ybins = np.logspace(0, 5, 30)

counts1, xedges, yedges = np.histogram2d(x1, y1, bins=(xbins, ybins))
counts2, xedges, yedges = np.histogram2d(x2, y2, bins=(xbins, ybins))

max_value = max(np.max(counts1), np.max(counts2))

#levels = np.arange(10, 60, 10)
levels = np.array([0.01*max_value, 0.3*max_value, 0.5*max_value])

cs1 = plt.contour(xedges[:-1], yedges[:-1], counts1.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='blue', label="TC")
ax.clabel(cs1, inline=False, fontsize=12)

cs2 = plt.contour(xedges[:-1], yedges[:-1], counts2.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='red', label="WT")
ax.clabel(cs2, inline=False, fontsize=12)

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlim(1e-2, 1e1)
ax.set_ylim(1e0, 1e5)

ax.set_yscale('log')
ax.set_xscale('log')

ax.set_xlabel(r"$\mathcal{E}$ energy", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

lines = [ cs1.collections[0], cs2.collections[0]]
labels = ["contained ({})".format(len(dfi)), "not contained ({})".format(len(dfo))]
ax.legend(lines, labels, prop={'size': 14}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title(r"$\mathcal{E}$ energy for " + WAVELET_LABEL, fontsize=20)

CCL for **Wavelets**:

1. as for Tailcut, less images at high energy for *contained images* (coherent previous ccl)
2. as for Tailcut, **better energy conservation for *contained images*** (why ?)

** Reminder **

$\mathcal{E}_{\text{energy}}$ (relative total counts difference) is defined by:

$$
\mathcal{E}_{\text{energy}}(\hat{\boldsymbol{S}}, \boldsymbol{S}^*) = \frac{ \text{abs} \left( \sum_i \hat{\boldsymbol{S}}_i - \sum_i \boldsymbol{S}^*_i \right) }{ \sum_i \boldsymbol{S}^*_i }
$$

with:

- $\hat{\boldsymbol{S}}$ the algorithm's output image (i.e. the *cleaned* image);
- $\boldsymbol{S}^*$ the reference image (i.e. the *clean* image).

## Delta psi

In [ ]:
# Plot delta psi #####################

def plot_ratio(ax, res_tuple_tc, res_tuple_wt):
    val_of_bins_tc, bins_tc, patches_tc = res_tuple_tc
    val_of_bins_wt, bins_wt, patches_wt = res_tuple_wt
    edges_of_bins = bins_tc

    # Set ratio where val_of_bins_data is not zero
    ratio = np.divide(val_of_bins_wt,
                      val_of_bins_tc,
                      where=(val_of_bins_tc != 0))

    # Compute error on ratio (null if cannot be computed)
    # This is wrong as it's made for Gaussian distributions and here we have Poisson distribution
    error = np.divide(val_of_bins_wt * np.sqrt(val_of_bins_tc) + val_of_bins_tc * np.sqrt(val_of_bins_wt),
                       np.power(val_of_bins_tc, 2),
                       where=(val_of_bins_tc != 0))

    ax.set_ylabel('Ratio (WT/TC)', fontsize=20)
    ax.axhline(y=1, linewidth=2, linestyle='--', color='gray', alpha=0.5)

    bincenter = 0.5 * (edges_of_bins[1:] + edges_of_bins[:-1])
    ax.errorbar(bincenter, ratio, yerr=error, fmt='o', color='k', elinewidth=3, capsize=4, capthick=3, linewidth=6)
    ax.plot(bincenter, ratio, 'ok', linewidth=6)

### Tailcut

In [ ]:
fig, ax = plt.subplots()

ax.plot(dfi['delta_tc'], dfi['peSum_tc'], '.b', alpha=0.1, label="contained ({})".format(len(dfi)))
ax.plot(dfo['delta_tc'], dfo['peSum_tc'], '.r', alpha=0.1, label="not contained ({})".format(len(dfo)))

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_yscale('log')

ax.set_xlabel("delta_psi", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title(r"$\Delta\psi$ for " + TAILCUT_LABEL, fontsize=20)

CCL: difficult to conclude anything... see the figure below

In [ ]:
fig, ax = plt.subplots()

x1, y1 = dfi['delta_tc'], dfi['peSum_tc']
x2, y2 = dfo['delta_tc'], dfo['peSum_tc']

xbins = np.linspace(0, 90, 50)
ybins = np.logspace(0, 5, 50)

counts1, xedges, yedges = np.histogram2d(x1, y1, bins=(xbins, ybins))
counts2, xedges, yedges = np.histogram2d(x2, y2, bins=(xbins, ybins))

max_value = max(np.max(counts1), np.max(counts2))

#levels = np.arange(10, 60, 10)
levels = np.array([0.1*max_value, 0.3*max_value, 0.4*max_value])

cs1 = plt.contour(xedges[:-1], yedges[:-1], counts1.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='blue', label="TC")
ax.clabel(cs1, inline=False, fontsize=12)

cs2 = plt.contour(xedges[:-1], yedges[:-1], counts2.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='red', label="WT")
ax.clabel(cs2, inline=False, fontsize=12)

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlim(0, 30)
ax.set_ylim(1e1, 1e5)

ax.set_yscale('log')
#ax.set_xscale('log')

ax.set_xlabel("delta_psi", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

lines = [ cs1.collections[0], cs2.collections[0]]
labels = ["contained ({})".format(len(dfi)), "not contained ({})".format(len(dfo))]
ax.legend(lines, labels, prop={'size': 14}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title(r"$\Delta\psi$ for " + TAILCUT_LABEL, fontsize=20)

CCL for **Tailcut**:

1. less images at high energy for *contained images* (coherent previous ccl)
2. seems to have slightly better precision for *contained images* around 100PE

### Wavelets

In [ ]:
fig, ax = plt.subplots()

# x=ref, y=clean
ax.plot(dfi['delta_wt'], dfi['peSum_wt'], '.b', alpha=0.1, label="contained ({})".format(len(dfi)))
ax.plot(dfo['delta_wt'], dfo['peSum_wt'], '.r', alpha=0.1, label="not contained ({})".format(len(dfo)))

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_yscale('log')

ax.set_xlabel("delta_psi", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

ax.legend(prop={'size': 11}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title(r"$\Delta\psi$ for " + WAVELET_LABEL, fontsize=20)

CCL: difficult to conclude anything... see the figure below

In [ ]:
fig, ax = plt.subplots()

x1, y1 = dfi['delta_wt'], dfi['peSum_wt']
x2, y2 = dfo['delta_wt'], dfo['peSum_wt']

xbins = np.linspace(0, 90, 50)
ybins = np.logspace(0, 5, 50)

counts1, xedges, yedges = np.histogram2d(x1, y1, bins=(xbins, ybins))
counts2, xedges, yedges = np.histogram2d(x2, y2, bins=(xbins, ybins))

max_value = max(np.max(counts1), np.max(counts2))

#levels = np.arange(10, 60, 10)
levels = np.array([0.1*max_value, 0.3*max_value, 0.4*max_value])

cs1 = plt.contour(xedges[:-1], yedges[:-1], counts1.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='blue', label="TC")
ax.clabel(cs1, inline=False, fontsize=12)

cs2 = plt.contour(xedges[:-1], yedges[:-1], counts2.T, levels,
                  linewidths=(2, 2, 3), linestyles=('dotted', 'dashed', 'solid'),
                  alpha=0.5, colors='red', label="WT")
ax.clabel(cs2, inline=False, fontsize=12)

ax.axhline(y=50, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=2000, linewidth=2, linestyle='--', color='black', alpha=0.5)
ax.axhline(y=FAINT_BRIGHT_BORDER, linewidth=2, linestyle=':', color='black', alpha=0.5)

ax.set_xlim(0, 30)
ax.set_ylim(1e1, 1e5)

ax.set_yscale('log')
#ax.set_xscale('log')

ax.set_xlabel("delta_psi", fontsize=20)
ax.set_ylabel("npe", fontsize=20)

lines = [ cs1.collections[0], cs2.collections[0]]
labels = ["contained ({})".format(len(dfi)), "not contained ({})".format(len(dfo))]
ax.legend(lines, labels, prop={'size': 14}, loc='best', fancybox=True, framealpha=0.5)

ax.set_title(r"$\Delta\psi$ for " + WAVELET_LABEL, fontsize=20)

CCL for Wavelets:

1. less images at high energy for *contained images* (coherent previous ccl)
2. seems to have slightly better precision for *contained images* around 100PE

In [ ]:
################################

fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(nrows=2, ncols=2, figsize=(16, 9))

#BINS_START, BINS_STOP, BINS_STEP = 0., 99., 9.
#BINS_START, BINS_STOP, BINS_STEP = 0., 99., 9.

#BINS = np.arange(BINS_START, BINS_STOP, BINS_STEP)
#BINS = 30
#BINS = np.linspace(BINS_START, BINS_STOP, 30)
#BINS = [3.53056780e-03, 5.99515231e+00, 1.19867741e+01, 1.79783958e+01, 2.39700176e+01, 2.99616393e+01,
#        3.59532610e+01, 4.19448828e+01, 4.79365045e+01, 5.39281263e+01, 5.99197480e+01, 6.59113698e+01,
#        7.19029915e+01, 7.78946133e+01, 8.38862350e+01, 8.98778568e+01]
BINS = 15

################################

NPE_MIN, NPE_MAX = 50, FAINT_BRIGHT_BORDER

df1 = dfi[dfi.peSum_wt > NPE_MIN][dfi.peSum_wt <= NPE_MAX]['delta_wt']
df2 = dfo[dfo.peSum_wt > NPE_MIN][dfo.peSum_wt <= NPE_MAX]['delta_wt']

res_tuple_tc1 = ax1.hist(df1,
                         label="contained ({})".format(len(df1)),
                         bins=BINS,
                         linewidth=2, alpha=.5, color='blue', histtype="step")

res_tuple_wt1 = ax1.hist(df2,
                         label="not contained ({})".format(len(df2)),
                         bins=BINS,
                         linewidth=2, alpha=.5, color='red',  histtype="step")

ax1.set_title("{} to {} NPE".format(NPE_MIN, NPE_MAX), fontsize=20)
ax1.set_xlabel("delta_psi", fontsize=20)
ax1.set_ylabel("counts", fontsize=20)
ax1.legend(prop={'size': 18}, loc='best', fancybox=True, framealpha=0.5)

plot_ratio(ax3, res_tuple_tc1, res_tuple_wt1)

################################

NPE_MIN, NPE_MAX = FAINT_BRIGHT_BORDER, 2000

df1 = dfi[dfi.peSum_wt > NPE_MIN][dfi.peSum_wt <= NPE_MAX]['delta_wt']
df2 = dfo[dfo.peSum_wt > NPE_MIN][dfo.peSum_wt <= NPE_MAX]['delta_wt']

res_tuple_tc2 = ax2.hist(df1,
                         label="contained ({})".format(len(df1)),
                         bins=BINS,
                         linewidth=2, alpha=.5, color='blue', histtype="step")

res_tuple_wt2 = ax2.hist(df2,
                         label="not contained ({})".format(len(df2)),
                         bins=BINS,
                         linewidth=2, alpha=.5, color='red',  histtype="step")

ax2.set_title("{} to {} NPE".format(NPE_MIN, NPE_MAX), fontsize=20)
ax2.set_xlabel("delta_psi", fontsize=20)
ax2.set_ylabel("counts", fontsize=20)
ax2.legend(prop={'size': 18}, loc='best', fancybox=True, framealpha=0.5)

plot_ratio(ax4, res_tuple_tc2, res_tuple_wt2)

#ax.set_yscale('log')

fig.suptitle(WAVELET_LABEL, fontsize=20)

Does this plot has any sense ?